In [547]:
import pandas as pd

# Tabla Proteins  
## debe tener las cols: *id_protein, uniprot_acc, hgnc_id, gene_id, gene_name, sequence, length*

In [548]:
box1 = pd.read_csv('box1_proteins.csv')
box1 = box1.rename(columns={'uniprot': 'uniprot_acc'})
box1 = box1.drop(columns='organism')

In [549]:
box1

,uniprot_acc,mlo,hgnc_id,length,sequence
0,O15527,nuclear speckle,HGNC:8125,345,MPARALLPRRMGHRTLASTPALWASIPCPRSELRLDLVLPSGQSFR...
1,O43663,pcg body,HGNC:9341,620,MRRSEVLAEESIVCLQKALNHLREIWELIGIPEDQRLQRTEVVKKH...
2,O75494,nuclear speckle,HGNC:16713,262,MSRYLRPPNTSLFVRNVADDTRSEDLRREFGRYGPIVDVYVPLDFY...
3,O95613,centrosome/spindle pole body,HGNC:16068,3336,MEVEQEQRRRKVEAGRTKLAHFRQRKTKGDSSHSEKKTAKRKGSAV...
4,P08047,centrosome/spindle pole body,HGNC:11205,785,MSDQDHSMDEMTAVVKIEKGVGGNNGGNGNGGGAFSQARSSSTGSS...
...,...,...,...,...,...
584,Q07352,cytoplasmic ribonucleoprotein granule,HGNC:1107,338,MTTTLVSATIFDLSEVLCKGNKMLNYSAPSAGGCLLDRKAVGTPAG...
585,Q8NE35,p-body,HGNC:21746,698,MQDDLLMDKSKTQPQPQQQQRQQQQPQPESSVSEAPSTPLSSETPK...
586,P26368,nuclear speckle,HGNC:23156,475,MSDFDEFERQLNENKQERDKENRHRKRSHSRSRSRDRKRRSRSRDR...
587,Q15233,paraspeckle,HGNC:7871,471,MQSNKTFNLEKQNHTPRKHHQHHHQQQHHQQQQQQPPPPPIPANGQ...


In [550]:
box1.uniprot_acc = box1.uniprot_acc.apply(str)
box1.hgnc_id = box1.hgnc_id.apply(str)

In [551]:
# tabla separada para MLOs
mlo = box1.copy()

## Cross references

In [552]:
# Esta tabla, obtenida en biomart, contiene entre otras cosas, gene_id, gene_name y uniprot_acc
bm = pd.read_csv('biomart_geneid_uniprot.csv')
bm = bm.rename(columns= {'Gene name': 'gene_name', 'HGNC ID': 'hgnc_id', 'UniProtKB/Swiss-Prot ID': 'uniprot_acc', 'NCBI gene (formerly Entrezgene) ID': 'gene_id'})
bm = bm[['uniprot_acc', 'hgnc_id', 'gene_name', 'gene_id']]
bm = bm.drop_duplicates()

In [553]:
bm[bm.gene_id.isnull()]

,uniprot_acc,hgnc_id,gene_name,gene_id
75,I0J062,NaN,AP006621.6,NaN
4149,Q8WZ33,NaN,AC145212.1,NaN
4309,Q969X6,HGNC:1983,UTP4,NaN


In [554]:
# Los agrego. Data obtenida de HGNC (genenames.org)
bm.loc[75, 'gene_name'] = 'PANO1'
bm.loc[75, 'hgnc_id'] = 'HGNC:51237'
bm.loc[75, 'gene_id'] = 101927423
bm.loc[4149, 'gene_name'] = 'MAFIP'
bm.loc[4149, 'hgnc_id'] = 'HGNC:31102'
bm.loc[4149, 'gene_id'] = 727764
bm.loc[4309, 'gene_id'] = 84916

In [555]:
bm.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 597 entries, 0 to 5912
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   uniprot_acc  597 non-null    object 
 1   hgnc_id      597 non-null    object 
 2   gene_name    597 non-null    object 
 3   gene_id      597 non-null    float64
dtypes: float64(1), object(3)
memory usage: 39.5+ KB


In [556]:
# Tabla con cross references customizada de HGNC (approved human gene nomenclature, genenames.org)
hgnc = pd.read_csv('hgnc_custom_all.txt', sep='\t')
hgnc = hgnc.rename(columns = {'HGNC ID': 'hgnc_id', 'UniProt ID(supplied by UniProt)': 'uniprot_acc', 'NCBI Gene ID': 'gene_id', 'Approved symbol': 'gene_name'})
# Subset de proteinas y estado 'Approved', dejo las cols que interesan
hgnc = hgnc[(hgnc.uniprot_acc.notnull()) & (hgnc.Status == 'Approved')]
hgnc = hgnc[['uniprot_acc', 'hgnc_id', 'gene_name', 'gene_id']]
hgnc

,uniprot_acc,hgnc_id,gene_name,gene_id
0,P04217,HGNC:5,A1BG,1.0
2,Q9NQ94,HGNC:24086,A1CF,29974.0
4,P01023,HGNC:7,A2M,2.0
6,A8K2U0,HGNC:23336,A2ML1,144568.0
12,U3KPV4,HGNC:30005,A3GALT2,127550.0
...,...,...,...,...
47678,Q6WRX3,HGNC:32058,ZYG11A,440590.0
47680,Q9C0D3,HGNC:25820,ZYG11B,79699.0
47681,Q15942,HGNC:13200,ZYX,7791.0
47683,O43149,HGNC:29027,ZZEF1,23140.0


In [557]:
hgnc.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 20256 entries, 0 to 47684
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   uniprot_acc  20256 non-null  object 
 1   hgnc_id      20256 non-null  object 
 2   gene_name    20256 non-null  object 
 3   gene_id      20097 non-null  float64
dtypes: float64(1), object(3)
memory usage: 791.2+ KB


In [558]:
# subseteo por todos los que no estan en bm, ya que bm esta completo
hgnc = hgnc[~hgnc.uniprot_acc.isin(bm.uniprot_acc)]

In [559]:
# Complemento ambas tablas en una
cross = pd.concat([hgnc, bm])

In [560]:
cross.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 20266 entries, 0 to 5912
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   uniprot_acc  20266 non-null  object 
 1   hgnc_id      20266 non-null  object 
 2   gene_name    20266 non-null  object 
 3   gene_id      20111 non-null  float64
dtypes: float64(1), object(3)
memory usage: 791.6+ KB


## Busco las proteinas de box1

In [561]:
aux = cross[cross.uniprot_acc.isin(box1.uniprot_acc)]
aux = aux.drop_duplicates(['uniprot_acc'])

In [562]:
aux.gene_name = aux.gene_name.apply(str)
aux.gene_id = aux.gene_id.apply(int)
aux.uniprot_acc = aux.uniprot_acc.apply(str)
aux.hgnc_id = aux.hgnc_id.apply(str)

In [563]:
aux[aux.uniprot_acc == 'I0J062']

,uniprot_acc,hgnc_id,gene_name,gene_id
75,I0J062,HGNC:51237,PANO1,101927423


In [564]:
aux.gene_id[aux.gene_id.isnull()]

Series([], Name: gene_id, dtype: int64)

In [565]:
aux.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 587 entries, 1034 to 5912
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   uniprot_acc  587 non-null    object
 1   hgnc_id      587 non-null    object
 2   gene_name    587 non-null    object
 3   gene_id      587 non-null    int64 
dtypes: int64(1), object(3)
memory usage: 22.9+ KB


In [566]:
box1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 589 entries, 0 to 588
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   uniprot_acc  589 non-null    object
 1   mlo          589 non-null    object
 2   hgnc_id      589 non-null    object
 3   length       589 non-null    int64 
 4   sequence     589 non-null    object
dtypes: int64(1), object(4)
memory usage: 23.1+ KB


In [567]:
box1 = box1.merge(aux, how='left')
box1 = box1.drop_duplicates(['uniprot_acc'])

In [568]:
box1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 589 entries, 0 to 588
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   uniprot_acc  589 non-null    object 
 1   mlo          589 non-null    object 
 2   hgnc_id      589 non-null    object 
 3   length       589 non-null    int64  
 4   sequence     589 non-null    object 
 5   gene_name    578 non-null    object 
 6   gene_id      578 non-null    float64
dtypes: float64(1), int64(1), object(5)
memory usage: 36.8+ KB


In [569]:
box1[box1.gene_id.isnull()]

,uniprot_acc,mlo,hgnc_id,length,sequence,gene_name,gene_id
18,Q86V81,nuclear speckle,nan,257,MADKMDMSLDDIIKLNRSQRGGRGGGRGRGRAGSQGGRGGGAQAAA...,NaN,NaN
70,P24928,nucleolus,nan,1970,MHGGGPPSGDSACPLRTIKRVQFGVLSPDELKRMSVTEGGIKYPET...,NaN,NaN
130,I0J062,nucleolus,nan,215,MGVRLRVWPAAPHSISRCPRPLGAVLSILLAGGSRKGTPTARCLGQ...,NaN,NaN
211,P78344,null_phasepdb_rev,nan,907,MESAIAEGGASRFSASSGGGGSRGAPQHYPKTAGNSEFLGKTPGQN...,NaN,NaN
224,Q0WX57,nucleolus,HGNC:44458,530,MEDDSLYLRGEWQFNHFSKLTSSRPDAAFAEIQRTSLPEKSPLSCE...,NaN,NaN
247,Q16198,transcription factories,nan,76,PTSYPPQTGSYSQAPSQYSQQSSSYGQQNPYQILGPTSSRLANPGS...,NaN,NaN
315,Q8WZ33,nucleolus,nan,124,MLCPRAAFLVGSFHGVFPGPPASSHWEFWVPSTPVGAYFCPPQPQL...,NaN,NaN
364,Q9BYB0,pre and postsynaptic density,nan,1731,MDGPGASAVVVRVGIPDLQQTKCLRLDPAAPVWAAKQRVLCALNHS...,NaN,NaN
411,Q9UN81,stress granule,nan,338,MGKKQNRKTGNSKTQSASPPPKERSSSPATEQSWMENDFDELREEG...,NaN,NaN
448,Q9UKL3,histone locus body,nan,1982,MAADDDNGDGTSLFDVFSASPLKNNDEGSLDIYAGLDSAVSDSASK...,NaN,NaN


In [570]:
aux[aux.uniprot_acc == 'Q86V81'] #xq no los agrega??

,uniprot_acc,hgnc_id,gene_name,gene_id
1034,Q86V81,HGNC:19071,ALYREF,10189


In [571]:
missing = (box1[box1.gene_id.isnull()]).uniprot_acc.to_list()

In [572]:
missing

['Q86V81',
 'P24928',
 'I0J062',
 'P78344',
 'Q0WX57',
 'Q16198',
 'Q8WZ33',
 'Q9BYB0',
 'Q9UN81',
 'Q9UKL3',
 'Q92841']

In [573]:
box1.gene_name = box1.gene_name.apply(str)
box1.gene_id = box1.gene_id.apply(float)
box1.uniprot_acc = box1.uniprot_acc.apply(str)
box1.hgnc_id = box1.hgnc_id.apply(str)

In [574]:
# los busque en uniprot
faltantes = pd.read_csv('uniprot_crossref_faltantes.tab.gz', sep= '\t', compression='gzip')
faltantes = faltantes.rename(columns = {'Entry': 'uniprot_acc', 'Gene names  (primary )': 'gene_name', 'Cross-reference (GeneID)': 'gene_id', 'Cross-reference (HGNC)': 'hgnc_id'})
faltantes = faltantes.drop(columns = ['Entry name', 'Status', 'Protein names', 'Organism'])

In [575]:
faltantes.gene_id = faltantes.gene_id.str.strip(';')
faltantes.hgnc_id = faltantes.hgnc_id.str.strip(';')
faltantes.hgnc_id = 'HGNC:' + faltantes.hgnc_id
faltantes.gene_id = faltantes.gene_id.apply(float)

In [576]:
faltantes.gene_name = faltantes.gene_name.apply(str)
faltantes.uniprot_acc = faltantes.uniprot_acc.apply(str)
faltantes.hgnc_id = faltantes.hgnc_id.apply(str)

In [577]:
faltantes

,uniprot_acc,gene_name,gene_id,hgnc_id
0,Q9BYB0,SHANK3,NaN,HGNC:14294
1,Q86V81,ALYREF,10189.0,HGNC:19071
2,P24928,POLR2A,5430.0,HGNC:9187
3,Q9UKL3,CASP8AP2,9994.0,HGNC:1510
4,Q92841,DDX17,10521.0,HGNC:2740
5,Q9UN81,L1RE1,NaN,nan
6,P78344,EIF4G2,1982.0,HGNC:3297
7,I0J062,PANO1,101927423.0,HGNC:51237
8,Q8WZ33,MAFIP,NaN,HGNC:31102
9,Q16198,EWS-Fli1,NaN,nan


In [578]:
box1 = box1.merge(faltantes, on=['uniprot_acc', 'gene_name', 'gene_id', 'hgnc_id'], how= 'left')

In [579]:
box1[box1.gene_id.isnull()]  # sigue sin agregarlos

,uniprot_acc,mlo,hgnc_id,length,sequence,gene_name,gene_id
18,Q86V81,nuclear speckle,nan,257,MADKMDMSLDDIIKLNRSQRGGRGGGRGRGRAGSQGGRGGGAQAAA...,nan,NaN
70,P24928,nucleolus,nan,1970,MHGGGPPSGDSACPLRTIKRVQFGVLSPDELKRMSVTEGGIKYPET...,nan,NaN
130,I0J062,nucleolus,nan,215,MGVRLRVWPAAPHSISRCPRPLGAVLSILLAGGSRKGTPTARCLGQ...,nan,NaN
211,P78344,null_phasepdb_rev,nan,907,MESAIAEGGASRFSASSGGGGSRGAPQHYPKTAGNSEFLGKTPGQN...,nan,NaN
224,Q0WX57,nucleolus,HGNC:44458,530,MEDDSLYLRGEWQFNHFSKLTSSRPDAAFAEIQRTSLPEKSPLSCE...,nan,NaN
247,Q16198,transcription factories,nan,76,PTSYPPQTGSYSQAPSQYSQQSSSYGQQNPYQILGPTSSRLANPGS...,nan,NaN
315,Q8WZ33,nucleolus,nan,124,MLCPRAAFLVGSFHGVFPGPPASSHWEFWVPSTPVGAYFCPPQPQL...,nan,NaN
364,Q9BYB0,pre and postsynaptic density,nan,1731,MDGPGASAVVVRVGIPDLQQTKCLRLDPAAPVWAAKQRVLCALNHS...,nan,NaN
411,Q9UN81,stress granule,nan,338,MGKKQNRKTGNSKTQSASPPPKERSSSPATEQSWMENDFDELREEG...,nan,NaN
448,Q9UKL3,histone locus body,nan,1982,MAADDDNGDGTSLFDVFSASPLKNNDEGSLDIYAGLDSAVSDSASK...,nan,NaN


In [580]:
box1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 589 entries, 0 to 588
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   uniprot_acc  589 non-null    object 
 1   mlo          589 non-null    object 
 2   hgnc_id      589 non-null    object 
 3   length       589 non-null    int64  
 4   sequence     589 non-null    object 
 5   gene_name    589 non-null    object 
 6   gene_id      578 non-null    float64
dtypes: float64(1), int64(1), object(5)
memory usage: 36.8+ KB


In [581]:
faltantes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   uniprot_acc  10 non-null     object 
 1   gene_name    10 non-null     object 
 2   gene_id      6 non-null      float64
 3   hgnc_id      10 non-null     object 
dtypes: float64(1), object(3)
memory usage: 448.0+ bytes


In [582]:
# Data obtenida de HGNC (genenames.org)
subset_cross_ref.gene_name[75] = 'PANO1'
subset_cross_ref.hgnc_id[75] = 'HGNC:51237'
subset_cross_ref.gene_name[4149] = 'MAFIP'
subset_cross_ref.hgnc_id[4149] = 'HGNC:31102'

<ipython-input-582-9953d855aefd>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subset_cross_ref.gene_name[75] = 'PANO1'
<ipython-input-582-9953d855aefd>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subset_cross_ref.hgnc_id[75] = 'HGNC:51237'
<ipython-input-582-9953d855aefd>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subset_cross_ref.gene_name[4149] = 'MAFIP'
<ipython-input-582-9953d855aefd>:5: SettingWithCopyWarning: 
A va

In [583]:
subset_cross_ref.gene_id = subset_cross_ref.gene_id.apply(int)

In [584]:
box1 = box1.merge(subset_cross_ref, how= 'left')

In [585]:
# Veo los nan que quedan
box1[box1.hgnc_id.isnull()]

,uniprot_acc,mlo,hgnc_id,length,sequence,gene_name,gene_id


In [586]:
# crear un rango para asignar un id tipo INT unique a cada proteina
range(1, len(box1))

range(1, 589)

In [587]:
# col id_mutation INT
box1['id_protein'] = range(1, len(box1)+1)

In [588]:
box1 = box1[['id_protein', 'uniprot_acc', 'hgnc_id', 'gene_id', 'gene_name', 'length', 'sequence']]

In [592]:
box1

,id_protein,uniprot_acc,hgnc_id,gene_id,gene_name,length,sequence
0,1,O15527,HGNC:8125,4968.0,OGG1,345,MPARALLPRRMGHRTLASTPALWASIPCPRSELRLDLVLPSGQSFR...
1,2,O43663,HGNC:9341,9055.0,PRC1,620,MRRSEVLAEESIVCLQKALNHLREIWELIGIPEDQRLQRTEVVKKH...
2,3,O75494,HGNC:16713,10772.0,SRSF10,262,MSRYLRPPNTSLFVRNVADDTRSEDLRREFGRYGPIVDVYVPLDFY...
3,4,O95613,HGNC:16068,5116.0,PCNT,3336,MEVEQEQRRRKVEAGRTKLAHFRQRKTKGDSSHSEKKTAKRKGSAV...
4,5,P08047,HGNC:11205,6667.0,SP1,785,MSDQDHSMDEMTAVVKIEKGVGGNNGGNGNGGGAFSQARSSSTGSS...
...,...,...,...,...,...,...,...
584,585,Q07352,HGNC:1107,677.0,ZFP36L1,338,MTTTLVSATIFDLSEVLCKGNKMLNYSAPSAGGCLLDRKAVGTPAG...
585,586,Q8NE35,HGNC:21746,22849.0,CPEB3,698,MQDDLLMDKSKTQPQPQQQQRQQQQPQPESSVSEAPSTPLSSETPK...
586,587,P26368,HGNC:23156,11338.0,U2AF2,475,MSDFDEFERQLNENKQERDKENRHRKRSHSRSRSRDRKRRSRSRDR...
587,588,Q15233,HGNC:7871,4841.0,NONO,471,MQSNKTFNLEKQNHTPRKHHQHHHQQQHHQQQQQQPPPPPIPANGQ...


In [591]:
box1.to_csv('datasets/proteins_table.csv', index= False)
# subir a https://github.com/alvaro-2/mutations